In [7]:
import cv2
import numpy as np
import os

In [19]:
# Load positive and negative samples
negative_images = 'Traffic-Dataset-1-Non-NN\\negative-train'
positive_images = 'Traffic-Dataset-1-Non-NN\\positive-train'
positive_samples = []  # List of paths to positive sample images
negative_samples = []  # List of paths to negative sample images

for filename in os.listdir(negative_images):
    image_path = os.path.join(negative_images, filename)
    negative_samples.append(image_path)

for filename in os.listdir(positive_images):
    image_path = os.path.join(positive_images, filename)
    positive_samples.append(image_path)

In [18]:
print(positive_samples)
print(negative_samples)

['Traffic-Dataset-1-Non-NN\\positive-train\\positive_0_hlp-01-20210610-205037-cut_mp4-43_jpg.rf.188b8666e5587af478fde58daa899fc0.jpg.jpg', 'Traffic-Dataset-1-Non-NN\\positive-train\\positive_100_hlp-01-20210610-205037-cut_mp4-41_jpg.rf.6fdaceb010b26d63ff7f301cb4216898.jpg.jpg', 'Traffic-Dataset-1-Non-NN\\positive-train\\positive_101_hlp-01-20210610-205037-cut_mp4-41_jpg.rf.6fdaceb010b26d63ff7f301cb4216898.jpg.jpg', 'Traffic-Dataset-1-Non-NN\\positive-train\\positive_102_hlp-01-20210610-205037-cut_mp4-41_jpg.rf.6fdaceb010b26d63ff7f301cb4216898.jpg.jpg', 'Traffic-Dataset-1-Non-NN\\positive-train\\positive_103_hlp-01-20210610-205037-cut_mp4-41_jpg.rf.6fdaceb010b26d63ff7f301cb4216898.jpg.jpg', 'Traffic-Dataset-1-Non-NN\\positive-train\\positive_104_hlp-01-20210610-205037-cut_mp4-41_jpg.rf.6fdaceb010b26d63ff7f301cb4216898.jpg.jpg', 'Traffic-Dataset-1-Non-NN\\positive-train\\positive_105_hlp-01-20210610-205037-cut_mp4-41_jpg.rf.6fdaceb010b26d63ff7f301cb4216898.jpg.jpg', 'Traffic-Dataset-1-No

In [20]:
win_size = (64, 64)  # Size of the detection window
block_size = (16, 16)  # Block size in pixels
block_stride = (8, 8)  # Block stride in pixels
cell_size = (8, 8)  # Cell size in pixels
num_bins = 9  # Number of bins in the histogram
hog_descriptor = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

In [36]:
neg_features = []
for img_path in negative_samples:
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    resized_img = cv2.resize(img, win_size)
    features = hog_descriptor.compute(resized_img)
    neg_features.append(features)
neg_features = np.array(neg_features, dtype=np.float32)
neg_labels = np.zeros((neg_features.shape[0], 1), dtype=np.int32)

In [37]:
pos_features = []
for img_path in positive_samples:
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    resized_img = cv2.resize(img, win_size)
    features = hog_descriptor.compute(resized_img)
    pos_features.append(features)
pos_features = np.array(pos_features, dtype=np.float32)
pos_labels = np.ones((pos_features.shape[0], 1), dtype=np.int32)

In [38]:
features = np.vstack((pos_features, neg_features))
labels = np.vstack((pos_labels, neg_labels))

In [39]:
print(labels)

[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]


In [40]:
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6))

In [41]:
rand = np.random.RandomState(123)
shuffle_indices = rand.permutation(len(features))
features, labels = features[shuffle_indices], labels[shuffle_indices]

In [43]:
svm.train(features, cv2.ml.ROW_SAMPLE, labels)
svm.save("hog_svm_model.xml")

print("HOG SVM model trained and saved.")

HOG SVM model trained and saved.
